<a href="https://colab.research.google.com/github/ddinesan/Manga/blob/master/Lecture_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HHL Algorithm


# Concepts
The HHL algorithm solves a system of linear equations, specifically of the form $\mathbf{A}x = b$, where $\mathbf{A}$ is a Hermitian matrix, $b$ is a known vector, and
$x$ is the unknown vector we want to find. 

When we solve this linear system on a quantum computer the solution is of the form:
$|x> = \frac{\mathbf{A}^{-1} |b>}{|| \mathbf{A}^{-1} |b> ||}$

The HHL algorithm uses 3 sets of qubits: a single ancillary qubit, a register qubits (to
store eigenvalues of $\mathbf{A}$), and a memory qubit (to store $|b>$) and is performed in the following order:

1) Quantum phase estimation to estimate eigenvalues of A

2) Controlled rotations of ancilla qubit

3) Uncomputation with inverse quantum phase estimation

The accuracy of the result depends on the following factors:
* Register size
* Choice of parameters $C$ and $t$
* Ideally we want eigenvalues of the matrix of the form
  $\frac{2\pi k}{t N}$,
  where $0 \leq k < N$, and $N=2^n$, for $n$ registers.
* $C \leq \frac{2\pi}{t N}$, is the smallest eigenvalue that
can be stored in the circuit. 

One way to set t is $t = \frac{2\pi}{sN}$, where $s$ is a scaling factor.

# Initial Setup

In [ ]:
# install cirq
!pip install cirq --quiet
import sys
import cirq
import sympy
import numpy as np
import scipy as sp
import math
import random

from cirq.contrib.svg import circuit_to_svg
from IPython.display import SVG, display
import io

%matplotlib inline
import matplotlib.pyplot

print('Installed python ', sys.version, '\ncirq ', cirq.__version__)

Installed python  3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0] 
cirq  0.7.0


# Circuit Diagram for Example


Example of circuit with 2 register qubits.


```
(0, 0): ─────────────────────────Ry(θ₄)─Ry(θ₁)─Ry(θ₂)─Ry(θ₃)──────────────M──
                     ┌──────┐    │      │      │      │ ┌───┐
(1, 0): ─H─@─────────│      │──X─@──────@────X─@──────@─│   │─────────@─H────
           │         │QFT^-1│    │      │      │      │ │QFT│         │
(2, 0): ─H─┼─────@───│      │──X─@────X─@────X─@────X─@─│   │─@───────┼─H────
           │     │   └──────┘                           └───┘ │       │
(3, 0): ─e^iAt──e^2iAt────────────────────────────────────e^-2iAt──e^-iAt──

```


Let's take a closer look at this circuit. The first line is our ancillary qubit, the next two lines are register qubits that are used to compute the eigenvalues of $\mathbf{A}$ and the last line is the memory that stores the known vector $|b>$. 

A lot of mechaninery is applied to the register qubits. It starts by getting the superposition of the registers. The next couple of gates with the $QFT^{-1}$ performs a phase estimation, which estimates the eigenvectors. The phase estimation is performed with hamiltonian simulation. Next we perform controlled y rotations, which is done to adjoin the different states. Finally we uncompute the phase estimation and measure the ancillary to get the Pauli observations of $|x>$.

## Cirq example with 4 register qubits

In [ ]:
class PhaseEstimation(cirq.Gate):
    """
    A gate for Quantum Phase Estimation.
    unitary is the unitary gate whose phases will be estimated.
    The last qubit stores the eigenvector; all other qubits store the
    estimated phase, in big-endian.
    """

    def __init__(self, num_qubits, unitary):
        self._num_qubits = num_qubits
        self.U = unitary

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        qubits = list(qubits)
        yield cirq.H.on_each(*qubits[:-1])
        yield PhaseKickback(self.num_qubits(), self.U)(*qubits)
        yield cirq.QFT(*qubits[:-1], without_reverse=True)**-1

class HamiltonianSimulation(cirq.EigenGate, cirq.SingleQubitGate):
    """
    A gate that represents e^iAt.
    This EigenGate + np.linalg.eigh() implementation is used here
    purely for demonstrative purposes. If a large matrix is used,
    the circuit should implement actual Hamiltonian simulation,
    by using the linear operators framework in Cirq for example.
    """

    def __init__(self, A, t, exponent=1.0):
        cirq.SingleQubitGate.__init__(self)
        cirq.EigenGate.__init__(self, exponent=exponent)
        self.A = A
        self.t = t
        ws, vs = np.linalg.eigh(A)
        self.eigen_components = []
        for w, v in zip(ws, vs.T):
            theta = w*t / math.pi
            P = np.outer(v, np.conj(v))
            self.eigen_components.append((theta, P))

    def _with_exponent(self, exponent):
        return HamiltonianSimulation(self.A, self.t, exponent)

    def _eigen_components(self):
        return self.eigen_components

class PhaseKickback(cirq.Gate):
    """
    A gate for the phase kickback stage of Quantum Phase Estimation.
    It consists of a series of controlled e^iAt gates with the memory qubit as
    the target and each register qubit as the control, raised
    to the power of 2 based on the qubit index.
    unitary is the unitary gate whose phases will be estimated.
    """

    def __init__(self, num_qubits, unitary):
        super(PhaseKickback, self)
        self._num_qubits = num_qubits
        self.U = unitary

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        qubits = list(qubits)
        memory = qubits.pop()
        for i, qubit in enumerate(qubits):
            yield cirq.ControlledGate(self.U**(2**i))(qubit, memory)

class EigenRotation(cirq.Gate):
    """
    EigenRotation performs the set of rotation on the ancilla qubit equivalent
    to division on the memory register by each eigenvalue of the matrix. The
    last qubit is the ancilla qubit; all remaining qubits are the register.
    It consists of a controlled ancilla qubit rotation for each possible value
    that can be represented by the register. Each rotation is a Ry gate where
    the angle is calculated from the eigenvalue corresponding to the register
    value, up to a normalization factor C.
    """

    def __init__(self, num_qubits, C, t):
        super(EigenRotation, self)
        self._num_qubits = num_qubits
        self.C = C
        self.t = t
        self.N = 2**(num_qubits-1)

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        for k in range(self.N):
            kGate = self._ancilla_rotation(k)

            # xor's 1 bits correspond to X gate positions.
            xor = k ^ (k-1)

            for q in qubits[-2::-1]:
                # Place X gates
                if xor % 2 == 1:
                    yield cirq.X(q)
                xor >>= 1

                # Build controlled ancilla rotation
                kGate = cirq.ControlledGate(kGate)

            yield kGate(*qubits)

    def _ancilla_rotation(self, k):
        if k == 0:
            k = self.N
        theta = 2*math.asin(self.C * self.N * self.t / (2*math.pi * k))
        return cirq.ry(theta)

def hhl_circuit(A, C, t, register_size, *input_prep_gates):
    """
    Constructs the HHL circuit.
    Args:
        A: The input Hermitian matrix.
        C: Algorithm parameter.
        t: Algorithm parameter.
        register_size: The size of the eigenvalue register.
        memory_basis: The basis to measure the memory in, one of 'x', 'y'.
        input_prep_gates: A list of gates to be applied to |0> to generate the
            desired input state |b>.
    Returns:
        The HHL circuit. The ancilla measurement has key 'a' and the memory
        measurement is in key 'm'.  There are two parameters in the circuit,
        `exponent` and `phase_exponent` corresponding to a possible rotation
        applied before the measurement on the memory with a
        `cirq.PhasedXPowGate`.
    """

    ancilla = cirq.LineQubit(0)
    # to store eigenvalues of the matrix
    register = [cirq.LineQubit(i + 1) for i in range(register_size)]
    # to store input and output vectors
    memory = cirq.LineQubit(register_size + 1)

    c = cirq.Circuit()
    hs = HamiltonianSimulation(A, t)
    pe = PhaseEstimation(register_size+1, hs)
    c.append([gate(memory) for gate in input_prep_gates])
    c.append([
        pe(*(register + [memory])),
        EigenRotation(register_size + 1, C, t)(*(register + [ancilla])),
        pe(*(register + [memory]))**-1,
        cirq.measure(ancilla, key='a')
    ])

    c.append([
        cirq.PhasedXPowGate(
            exponent=sympy.Symbol('exponent'),
            phase_exponent=sympy.Symbol('phase_exponent'))(memory),
        cirq.measure(memory, key='m')
    ])

    return c

def simulate(circuit):
    simulator = cirq.Simulator()

    # Cases for measurring X, Y, and Z (respectively) on the memory qubit.
    params = [{
        'exponent': 0.5,
        'phase_exponent': -0.5
    }, {
        'exponent': 0.5,
        'phase_exponent': 0
    }, {
        'exponent': 0,
        'phase_exponent': 0
    }]

    results = simulator.run_sweep(circuit, params, repetitions=5000)

    for label, result in zip(('X', 'Y', 'Z'), list(results)):
        # Only select cases where the ancilla is 1.
        expectation = 1 - 2 * np.mean(
            result.measurements['m'][result.measurements['a'] == 1])
        print('{} = {}'.format(label, expectation))

def main():
    """
    Simulates HHL with matrix input, and outputs Pauli observables of the
    resulting qubit state |x>.
    Expected observables are calculated from the expected solution |x>.
    """

    # Eigendecomposition:
    #   (4.537, [-0.971555, -0.0578339+0.229643j])
    #   (0.349, [-0.236813, 0.237270-0.942137j])
    # |b> = (0.64510-0.47848j, 0.35490-0.47848j)
    # |x> = (-0.0662724-0.214548j, 0.784392-0.578192j)
    A = np.array([[4.30213466-6.01593490e-08j,
                   0.23531802+9.34386156e-01j],
                  [0.23531882-9.34388383e-01j,
                   0.58386534+6.01593489e-08j]])
    t = 0.358166*math.pi
    register_size = 4
    input_prep_gates = [cirq.rx(1.276359), cirq.rz(1.276359)]
    expected = (0.144130, 0.413217, -0.899154)

    # Set C to be the smallest eigenvalue that can be represented by the
    # circuit.
    C = 2*math.pi / (2**register_size * t)

    # Simulate circuit
    print("Expected observable outputs:")
    print("X =", expected[0])
    print("Y =", expected[1])
    print("Z =", expected[2],"\n")
    print("Simulated: ")
    simulate(hhl_circuit(A, C, t, register_size, *input_prep_gates))


if __name__ == '__main__':
    main()

Expected observable outputs:
X = 0.14413
Y = 0.413217
Z = -0.899154 

Simulated: 
X = 0.17961165048543692
Y = 0.41896938013442864
Z = -0.8785992217898833


Code Source: cirq/examples/hhl, Git hub repository: https://github.com/quantumlib/Cirq/blob/master/examples/hhl.py#L132

# Solving HHL with Householder Unitary

Theoretically, phase estimation can be performed with any unitary matrix as shown in [3]. So I wanted to see what the result would look like using a householder unitary matrix $\mathbf{U}$. The circuit does not change too much. We just simply replace $e^{i\mathbf{A}t}$ with a householder unitary given by:
$\mathbf{U} = \mathbf{I} - 2|\psi><\psi|$

# Circuit using Householder Unitary

Example of circuit with 2 register qubits.


```
(0, 0): ─────────────────────────Ry(θ₄)─Ry(θ₁)─Ry(θ₂)─Ry(θ₃)──────────────M──
                     ┌──────┐    │      │      │      │ ┌───┐
(1, 0): ─H─@─────────│      │──X─@──────@────X─@──────@─│   │─────────@─H────
           │         │QFT^-1│    │      │      │      │ │QFT│         │
(2, 0): ─H─┼─────@───│      │──X─@────X─@────X─@────X─@─│   │─@───────┼─H────
           │     │   └──────┘                           └───┘ │       │
(3, 0): ── U ── U^2 ────────────────────────────────────────── U ─── U^2 ────

```

# Example with Householder Unitary



In [ ]:
class PhaseEstimation(cirq.Gate):
    """
    A gate for Quantum Phase Estimation.
    unitary is the unitary gate whose phases will be estimated.
    The last qubit stores the eigenvector; all other qubits store the
    estimated phase, in big-endian.
    """

    def __init__(self, num_qubits, unitary):
        self._num_qubits = num_qubits
        self.U = unitary

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        qubits = list(qubits)
        yield cirq.H.on_each(*qubits[:-1])
        yield PhaseKickback(self.num_qubits(), self.U)(*qubits)
        yield cirq.QFT(*qubits[:-1], without_reverse=True)**-1

class Householder(cirq.EigenGate, cirq.SingleQubitGate):
    """
    A gate that represents the householder unitary matrix.
    """

    def __init__(self, A, t, exponent=1.0):
        cirq.SingleQubitGate.__init__(self)
        cirq.EigenGate.__init__(self, exponent=exponent)
        self.A = A
        self.t = t
        ws, vs = sp.linalg.eigh(A)
        self.eigen_components = []
        for w, v in zip(ws, vs.T):
            theta = w*t / math.pi
            P = np.eye(v.size) - 2 * np.outer(v, np.conj(v))
            self.eigen_components.append((theta, P))

    def _with_exponent(self, exponent):
        return Householder(self.A, self.t, exponent)

    def _eigen_components(self):
        return self.eigen_components

class PhaseKickback(cirq.Gate):
    """
    A gate for the phase kickback stage of Quantum Phase Estimation.
    It consists of a series of controlled householder gates with the memory qubit as
    the target and each register qubit as the control.
    unitary is the unitary gate whose phases will be estimated.
    """

    def __init__(self, num_qubits, unitary):
        super(PhaseKickback, self)
        self._num_qubits = num_qubits
        self.U = unitary

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        qubits = list(qubits)
        memory = qubits.pop()
        for i, qubit in enumerate(qubits):
            yield cirq.ControlledGate(self.U**(2**i))(qubit, memory)

class EigenRotation(cirq.Gate):
    """
    EigenRotation performs the set of rotation on the ancilla qubit equivalent
    to division on the memory register by each eigenvalue of the matrix. The
    last qubit is the ancilla qubit.
    It consists of a controlled ancilla qubit rotation for each possible value
    that can be represented by the register. Each rotation is a Ry gate where
    the angle is calculated from the eigenvalue corresponding to the register
    value, up to a normalization factor C.
    """

    def __init__(self, num_qubits, C, t):
        super(EigenRotation, self)
        self._num_qubits = num_qubits
        self.C = C
        self.t = t
        self.N = 2**(num_qubits-1)

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        for k in range(self.N):
            kGate = self._ancilla_rotation(k)

            # xor's 1 bits correspond to X gate positions.
            xor = k ^ (k-1)

            for q in qubits[-2::-1]:
                # Place X gates
                if xor % 2 == 1:
                    yield cirq.X(q)
                xor >>= 1

                # Build controlled ancilla rotation
                kGate = cirq.ControlledGate(kGate)

            yield kGate(*qubits)

    def _ancilla_rotation(self, k):
        if k == 0:
            k = self.N
        theta = 2*math.asin(self.C * self.N * self.t / (2*math.pi * k))
        return cirq.ry(theta)

def hhl_circuit(A, C, t, register_size, *input_prep_gates):
    """
    Constructs the HHL circuit.
    Args:
        A: The input Hermitian matrix.
        C: Algorithm parameter.
        t: Algorithm parameter.
        register_size: The size of the eigenvalue register.
        memory_basis: The basis to measure the memory in, one of 'x', 'y', 'z'.
        input_prep_gates: A list of gates to be applied to |0> to generate the
            desired input state |b>.
    Returns:
        The HHL circuit. The ancilla measurement has key 'a' and the memory
        measurement is in key 'm'.
    """

    ancilla = cirq.LineQubit(0)
    # to store eigenvalues of the matrix
    register = [cirq.LineQubit(i + 1) for i in range(register_size)]
    # to store input and output vectors
    memory = cirq.LineQubit(register_size + 1)

    c = cirq.Circuit()
    hs = Householder(A, t)
    pe = PhaseEstimation(register_size + 1, hs)
    c.append([gate(memory) for gate in input_prep_gates])
    c.append([
        pe(*(register + [memory])),
        EigenRotation(register_size + 1, C, t)(*(register + [ancilla])),
        pe(*(register + [memory])),
        cirq.measure(ancilla, key='a')
    ])

    c.append([
        cirq.PhasedXPowGate(
            exponent=sympy.Symbol('exponent'),
            phase_exponent=sympy.Symbol('phase_exponent'))(memory),
        cirq.measure(memory, key='m')
    ])

    return c

def simulate(circuit):
    simulator = cirq.Simulator()

    # Cases for measuring X, Y, and Z (respectively) on the memory qubit.
    params = [{
        'exponent': 0.5,
        'phase_exponent': -0.5
    }, {
        'exponent': -0.5,
        'phase_exponent': 0
    }, {
        'exponent': 1,
        'phase_exponent': 0
    }]

    results = simulator.run_sweep(circuit, params, repetitions=5000)

    for label, result in zip(('X', 'Y', 'Z'), list(results)):
        # Only select cases where the ancilla is 1.
        expectation = 1 - 2 * np.mean(
            result.measurements['m'][result.measurements['a'] == 1])
        print('{} = {}'.format(label, expectation))

def main():
    """
    Simulates HHL with matrix input, and outputs Pauli observables of the
    resulting qubit state |x>.
    Expected observables are calculated from the expected solution |x>.
    """

    # Eigendecomposition:
    #   (4.537, [-0.971555, -0.0578339+0.229643j])
    #   (0.349, [-0.236813, 0.237270-0.942137j])
    # |b> = (0.64510-0.47848j, 0.35490-0.47848j)
    # |x> = (-0.0662724-0.214548j, 0.784392-0.578192j)
    A = np.array([[4.30213466-6.01593490e-08j,
                   0.23531802+9.34386156e-01j],
                  [0.23531882-9.34388383e-01j,
                   0.58386534+6.01593489e-08j]])
    t = 0.358166*math.pi
    register_size = 4
    input_prep_gates = [cirq.rx(1.276359), cirq.rz(1.276359)]
    expected = (0.144130, 0.413217, -0.899154)

    # Set C to be the smallest eigenvalue that can be represented by the
    # circuit.
    C = 2*math.pi / (2**register_size * t)

    # Simulate circuit
    print("Expected observable outputs:")
    print("X =", expected[0])
    print("Y =", expected[1])
    print("Z =", expected[2],"\n")
    print("Simulated: ")
    simulate(hhl_circuit(A, C, t, register_size, *input_prep_gates))


if __name__ == '__main__':
    main()

Expected observable outputs:
X = 0.14413
Y = 0.413217
Z = -0.899154 

Simulated: 
X = 0.12798874824191275
Y = 0.5595153243050606
Z = -0.8272327964860908


By using the householder unitary matrix we are able to get similar performance to Hamiltonian simulation. This shows that we don't have to be limited to specific unitary matrices when solving HHL.

Extensions to HHL actually drop phase estimation entirely and use amplitude amplification instead.

#References

[1] [HHL algorithm github](https://github.com/quantumlib/Cirq/blob/master/examples/hhl.py)

[2] [HHL background](https://www.cs.umd.edu/class/fall2018/cmsc657/note/HHL.pdf)

[3] [Quantum Phase Estimation](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html)
